In [2]:
import nltk
import string

from os import listdir
from os.path import isfile, join

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
INPUT_DIR = "./data"

## Preprocessing

In [4]:
def returnListOfFilePaths(folderPath):
    fileInfo = []
    listOfFileNames = [fileName for fileName in listdir(folderPath) if isfile(join(folderPath, fileName))]
    listOfFilePaths = [join(folderPath, fileName) for fileName in listdir(folderPath) if isfile(join(folderPath, fileName))]
    fileInfo.append(listOfFileNames)
    fileInfo.append(listOfFilePaths)
    return fileInfo

fileNames, filePaths = returnListOfFilePaths(INPUT_DIR)
print("File Names:", fileNames, "\n", "File Paths:" ,filePaths)

File Names: ['news1.txt', 'news2.txt', 'news3.txt'] 
 File Paths: ['./data\\news1.txt', './data\\news2.txt', './data\\news3.txt']


In [5]:
# Get document contents
def create_docContentDict(filePaths):
    rawContentDict = {}
    for filePath in filePaths:
        with open(filePath, "r") as ifile:
            fileContent = ifile.read()
        rawContentDict[filePath] = fileContent
    return rawContentDict
rawContentDict = create_docContentDict(filePaths)
print(rawContentDict)

{'./data\\news1.txt': 'Channel tunnel operator Eurotunnel on Monday announced details of a deal giving bank creditors 45.5 percent of the company in return for wiping out 1.0 billion pounds ($1.6 billion) of its massive debts.\nThe long-awaited but highly complex restructuring of nearly nearly nine billion pounds of debt and unpaid interest throws the company a lifeline which could secure what is still likely to be a difficult future.\nThe deal, announced simultaneously in Paris and London, brings the company back from the brink of bankruptcy but leaves current shareholders, who have already seen their investment dwindle, owning only 54.5 percent of the company.\n"We have fixed and capped the interest payments and arranged only to pay what is available in cash," Eurotunnel co-chairman Alastair Morton told reporters at a news conference. "Avoiding having to do this again is the name of the game."\nMorton said the plan provides the Anglo-French company with the medium term financial stab

## Custom Tokenizer

Steps for tokenization:
- tokenize the input
- remove stop words
- perform stemming
- remove punctuations
- lowercase

In [6]:
def tokenizeContent(contentsRaw):
    tokenized = nltk.tokenize.word_tokenize(contentsRaw)
    return tokenized

In [7]:
def removeStopWordsFromTokenized(contentsTokenized):
    stop_word_set = set(nltk.corpus.stopwords.words("english"))
    filteredContents = [word for word in contentsTokenized if word not in stop_word_set]
    return filteredContents

In [8]:
def performPorterStemmingOnContents(contentsTokenized):
    porterStemmer = nltk.stem.PorterStemmer()
    filteredContents = [porterStemmer.stem(word) for word in contentsTokenized]
    return filteredContents

In [9]:
def removePunctuationFromTokenized(contentsTokenized):
    excludePuncuation = set(string.punctuation)
    
    # manually add additional punctuation to remove. This is just a sample
    doubleSingleQuote = '\'\''
    doubleDash = '--'
    doubleTick = '``'

    excludePuncuation.add(doubleSingleQuote)
    excludePuncuation.add(doubleDash)
    excludePuncuation.add(doubleTick)

    filteredContents = [word for word in contentsTokenized if word not in excludePuncuation]
    return filteredContents

In [10]:
def convertItemsToLower(contentsRaw):
    filteredContents = [term.lower() for term in contentsRaw]
    return filteredContents

In [11]:
content_test = rawContentDict[filePaths[0]]
print("Raw Content:", content_test[:300])

Raw Content: Channel tunnel operator Eurotunnel on Monday announced details of a deal giving bank creditors 45.5 percent of the company in return for wiping out 1.0 billion pounds ($1.6 billion) of its massive debts.
The long-awaited but highly complex restructuring of nearly nearly nine billion pounds of debt a


In [12]:
content_test_tokenized = tokenizeContent(content_test)
print("Tokenized content:", content_test_tokenized[:30])

Tokenized content: ['Channel', 'tunnel', 'operator', 'Eurotunnel', 'on', 'Monday', 'announced', 'details', 'of', 'a', 'deal', 'giving', 'bank', 'creditors', '45.5', 'percent', 'of', 'the', 'company', 'in', 'return', 'for', 'wiping', 'out', '1.0', 'billion', 'pounds', '(', '$', '1.6']


In [13]:
nltk.download('stopwords')
content_test_rmStop = removeStopWordsFromTokenized(content_test_tokenized)
print("After stop word removal:", content_test_rmStop[:30])

After stop word removal: ['Channel', 'tunnel', 'operator', 'Eurotunnel', 'Monday', 'announced', 'details', 'deal', 'giving', 'bank', 'creditors', '45.5', 'percent', 'company', 'return', 'wiping', '1.0', 'billion', 'pounds', '(', '$', '1.6', 'billion', ')', 'massive', 'debts', '.', 'The', 'long-awaited', 'highly']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Karishma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
content_test_stemmed = performPorterStemmingOnContents(content_test_rmStop)
print("After stemming:", content_test_stemmed[:30])

After stemming: ['channel', 'tunnel', 'oper', 'eurotunnel', 'monday', 'announc', 'detail', 'deal', 'give', 'bank', 'creditor', '45.5', 'percent', 'compani', 'return', 'wipe', '1.0', 'billion', 'pound', '(', '$', '1.6', 'billion', ')', 'massiv', 'debt', '.', 'the', 'long-await', 'highli']


In [15]:
content_test_cleaned = removePunctuationFromTokenized(content_test_stemmed)
content_test_clean_lower = convertItemsToLower(content_test_cleaned)
print("After lowercasing:", content_test_clean_lower[:30])

After lowercasing: ['channel', 'tunnel', 'oper', 'eurotunnel', 'monday', 'announc', 'detail', 'deal', 'give', 'bank', 'creditor', '45.5', 'percent', 'compani', 'return', 'wipe', '1.0', 'billion', 'pound', '1.6', 'billion', 'massiv', 'debt', 'the', 'long-await', 'highli', 'complex', 'restructur', 'nearli', 'nearli']


In [16]:
def processData(rawContents):
    cleaned = tokenizeContent(rawContents)
    cleaned = removeStopWordsFromTokenized(cleaned)
    cleaned = performPorterStemmingOnContents(cleaned)    
    cleaned = removePunctuationFromTokenized(cleaned)
    cleaned = convertItemsToLower(cleaned)
    return cleaned

## Output
Handles both TFIDF and Cosine Similarity 

In [17]:
# print TFIDF values in 'table' format
def print_TFIDF_for_all(term, values, fileNames):
    values = values.transpose() # files along 'x-axis', terms along 'y-axis'
    numValues = len(values[0])
    print('                ', end="")   #bank space for formatting output
    for n in range(len(fileNames)):
        print('{0:18}'.format(fileNames[n]), end="")    #file names
    print()
    for i in range(len(term)):
        print('{0:8}'.format(term[i]), end='\t|  ')     #the term
        for j in range(numValues):
            print('{0:.12f}'.format(values[i][j]), end='   ') #the value, corresponding to the file name, for the term
        print()

In [18]:
# write TFIDF values in 'table' format
def write_TFIDF_for_all(term, values, fileNames):
    filePath = "../results/tfid.txt"
    outFile = open(filePath, 'a')
    title = "TFIDF\n"
    outFile.write(title)
    values = values.transpose() # files along 'x-axis', terms along 'y-axis'
    numValues = len(values[0])
    outFile.write('               \t')   #bank space for formatting output
    for n in range(len(fileNames)):
        outFile.write('{0:18}'.format(fileNames[n]))    #file names
    outFile.write("\n")
    for i in range(len(term)):
        outFile.write('{0:15}'.format(term[i]))     #the term
        outFile.write('\t|  ')
        for j in range(numValues):
            outFile.write('{0:.12f}'.format(values[i][j])) #the value, corresponding to the file name, for the term
            outFile.write('   ')
        outFile.write("\n")

    outFile.close()

In [19]:
# TODO: modify this to build matrix then print from matrix form
def calc_and_print_CosineSimilarity_for_all(tfs, fileNames):
    #print(cosine_similarity(tfs[0], tfs[1]))
    print("\n\n\n========COSINE SIMILARITY====================================================================\n")
    numFiles = len(fileNames)
    names = []
    print('                   ', end="")    #formatting
    for i in range(numFiles):
        if i == 0:
            for k in range(numFiles):
                print(fileNames[k], end='   ')
            print()

        print(fileNames[i], end='   ')
        for n in range(numFiles):
            #print(fileNames[n], end='\t')
            matrixValue = cosine_similarity(tfs[i], tfs[n])
            numValue = matrixValue[0][0]
            #print(numValue, end='\t')
            names.append(fileNames[n])
            print(" {0:.8f}".format(numValue), end='         ')
            #(cosine_similarity(tfs[i], tfs[n]))[0][0]

        print()
    print("\n\n=============================================================================================\n")

In [20]:
def calc_and_write_CosineSimilarity_for_all(tfs, fileNames):
    filePath = "../results/cosine_similarity.txt"
    outFile = open(filePath, 'a')
    title = "COSINE SIMILARITY\n"
    outFile.write(title)
    numFiles = len(fileNames)
    names = []
    outFile.write('                   ')
    for i in range(numFiles):
        if i == 0:
            for k in range(numFiles):
                outFile.write(fileNames[k])
                outFile.write('   ')
            outFile.write("\n")
        outFile.write(fileNames[i])
        outFile.write('   ')

        for n in range(numFiles):
            matrixValue = cosine_similarity(tfs[i], tfs[n])
            numValue = matrixValue[0][0]
            names.append(fileNames[n])
            outFile.write('{0:.8f}'.format(numValue))
            outFile.write('         ')
            #(cosine_similarity(tfs[i], tfs[n]))[0][0]

        outFile.write("\n")

    outFile.close()

In [21]:
def main(printResults=True):
    baseFolderPath = "./data/"

    fileNames, filePathList = returnListOfFilePaths(baseFolderPath)

    rawContentDict = create_docContentDict(filePathList)

    # calculate tfidf
    tfidf = TfidfVectorizer(tokenizer=processData, stop_words='english')
    tfs = tfidf.fit_transform(rawContentDict.values())
    tfs_Values = tfs.toarray()
    tfs_Term = tfidf.get_feature_names()
    
    if printResults:
        # print results
        print_TFIDF_for_all(tfs_Term, tfs_Values, fileNames)
        calc_and_print_CosineSimilarity_for_all(tfs, fileNames)
    else:
        # write results to file
        write_TFIDF_for_all(tfs_Term, tfs_Values, fileNames)   
        calc_and_write_CosineSimilarity_for_all(tfs, fileNames)

In [22]:
main()

c:\users\karishma\appdata\local\programs\python\python39\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afterward', 'alon', 'alreadi', 'alway', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becom', 'besid', 'cri', 'describ', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'otherwis', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'thenc', 'thereaft', 'therebi', 'therefor', 'thu', 'togeth', 'twelv', 'twenti', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev'] not in stop_words.
  warnings.warn(
c:\users\karishma\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\d

                news1.txt         news2.txt         news3.txt         
'm      	|  0.083419176409   0.000000000000   0.000000000000   
's      	|  0.098537418799   0.148980349723   0.102134882092   
1.0     	|  0.031721216913   0.038367865097   0.000000000000   
1.56    	|  0.000000000000   0.100898263637   0.000000000000   
1.6     	|  0.041709588205   0.000000000000   0.000000000000   
1.85    	|  0.041709588205   0.000000000000   0.000000000000   
10      	|  0.031721216913   0.038367865097   0.000000000000   
10.40   	|  0.031721216913   0.038367865097   0.000000000000   
100     	|  0.000000000000   0.000000000000   0.043232346913   
113.5   	|  0.031721216913   0.038367865097   0.000000000000   
13.6    	|  0.000000000000   0.050449131818   0.000000000000   
130     	|  0.031721216913   0.038367865097   0.000000000000   
14.1    	|  0.000000000000   0.050449131818   0.000000000000   
150     	|  0.031721216913   0.000000000000   0.032879314160   
160     	|  0.031721216913   0.03

includ  	|  0.000000000000   0.000000000000   0.129697040738   
incom   	|  0.000000000000   0.000000000000   0.043232346913   
ingeni  	|  0.041709588205   0.000000000000   0.000000000000   
insensit	|  0.000000000000   0.000000000000   0.043232346913   
insolv  	|  0.000000000000   0.050449131818   0.000000000000   
instruct	|  0.000000000000   0.050449131818   0.000000000000   
instrument	|  0.031721216913   0.038367865097   0.000000000000   
intent  	|  0.000000000000   0.000000000000   0.043232346913   
interim 	|  0.000000000000   0.000000000000   0.043232346913   
invest  	|  0.063442433826   0.000000000000   0.098637942481   
issu    	|  0.095163650740   0.038367865097   0.000000000000   
john    	|  0.000000000000   0.000000000000   0.043232346913   
june    	|  0.000000000000   0.000000000000   0.043232346913   
larg    	|  0.000000000000   0.000000000000   0.129697040738   
late    	|  0.000000000000   0.050449131818   0.000000000000   
later   	|  0.000000000000   0.0000000

sustain 	|  0.041709588205   0.000000000000   0.000000000000   
swap    	|  0.063442433826   0.038367865097   0.000000000000   
syndic  	|  0.000000000000   0.050449131818   0.000000000000   
taken   	|  0.024634354700   0.029796069945   0.025533720523   
talk    	|  0.000000000000   0.000000000000   0.043232346913   
term    	|  0.041709588205   0.000000000000   0.000000000000   
throw   	|  0.041709588205   0.000000000000   0.000000000000   
time    	|  0.031721216913   0.000000000000   0.032879314160   
told    	|  0.083419176409   0.000000000000   0.000000000000   
total   	|  0.000000000000   0.000000000000   0.043232346913   
trade   	|  0.031721216913   0.038367865097   0.000000000000   
tradit  	|  0.000000000000   0.000000000000   0.043232346913   
transit 	|  0.000000000000   0.000000000000   0.086464693825   
treat   	|  0.000000000000   0.000000000000   0.043232346913   
tsb     	|  0.000000000000   0.000000000000   0.043232346913   
tuesday 	|  0.031721216913   0.038367865